In [2]:
import pandas as pd
import os
from glob import glob
from collections import defaultdict
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import re

In [21]:
# Funzione per aprire i file in una directory e ordinarli
def open_files(directory):
    files = os.listdir(directory)
    files.sort()
    return files

In [30]:
import os
import re
import pandas as pd

model_files = open_files('models_generations')
actual_files = open_files('actuals')

models = {}
for model_file in model_files:
    model_name = re.sub(r"^(fine-tuned-)?|-decoding-\d+\.csv$", "", model_file)
    print(f"Processing model: {model_name}")

    if 'qwen' in model_name.lower():
        print(f"Skipping model '{model_name}' as it contains 'qwen'.")
        continue

    model = pd.read_csv(os.path.join('models_generations', model_file))

    for actual_file in actual_files:
        if "global" in actual_file:
            actual = pd.read_csv(os.path.join('actuals', actual_file))

            merged = pd.merge(actual, model, left_on='index', right_index=True, how='left')

            # Assicuriamoci che 'input' sia mantenuto
            #if 'input' not in merged.columns and 'input' in model.columns:
            #    merged['input'] = model['input']

            merged_file = f"{model_name}_merged_output.csv"
            merged.to_csv(merged_file, index=False)

            # ritornami i record in cui la prediciton è nan
            nan_predictions = merged[merged['prediction'].isna()]
            print(f"Found {len(nan_predictions)} NaN predictions in {model_name} for {actual_file}.")  
            if not nan_predictions.empty:
                print(nan_predictions)


Processing model: LLaMAntino-3-ANITA-8B-Inst-DPO-ITA_predictions.csv
Found 0 NaN predictions in LLaMAntino-3-ANITA-8B-Inst-DPO-ITA_predictions.csv for global_actual_labels.csv.
Processing model: Llama-3.1-8B-Instruct_predictions.csv
Found 14 NaN predictions in Llama-3.1-8B-Instruct_predictions.csv for global_actual_labels.csv.
     index             post_id_x            reply_id_x               actual  \
6        6  18184193230015000576   3263700000000000000                OTHER   
10      10   8374295896122236928   9960700000000000000      FALSE ASSERTION   
19      19   1894119167367809024   9033280000000000000                OTHER   
29      29  14374185223286280192  15962200000000000000            HYPERBOLE   
68      68  16300389213436499968   7681940000000000000                OTHER   
86      86   3037383543302795776   6283640000000000000        CONTEXT SHIFT   
88      88   9408805770120683520   2035190000000000000            HYPERBOLE   
95      95  11534803039351519232  14401